## 微信账单到随手记

### 1.获取微信账单
1. 微信-我-设置-帮助与反馈
2. 支付-账户问题-微信支付如何导出账单-点击这里-个人对账下载

### 2.简单处理账单文件
1. 去掉文档前面的描述性字符
2. 将文档编码改为utf-8

### 3.运行Python代码
1. 将wechat_filepath的值更新为最新的导出文件
2. 执行代码得到wechat_output.xls文件

### 4.导入到随手记
1. 打开web端地址：https://www.sui.com/data/index.jsp
2. 新功能-导入导出-数据导入-随手记web版
3. 选择文件-上传数据

In [1]:
#原始代码来自此文档：https://blog.csdn.net/luchengtao11/article/details/127340578

from tkinter.messagebox import NO
import xlrd
import xlwt
from xlutils.copy import copy
from datetime import date, datetime
 
import codecs
import csv

In [2]:
template_filepath = 'template_update.xls'
wechat_filepath = '微信支付账单(20230806-20230909).csv'

In [3]:
read_book = xlrd.open_workbook(template_filepath, formatting_info=False)

print('所有的工作表：', read_book.sheet_names())
r_expense_sheet = read_book.sheet_names()[0]

r_expense_sheet = read_book.sheet_by_index(0)
 
print('工作表名称：%s，行数：%d，列数：%d' %
      (r_expense_sheet.name, r_expense_sheet.nrows, r_expense_sheet.ncols))
head: list = r_expense_sheet.row_values(0)
index = head.index('交易类型')
 
write_book = copy(read_book)
expense_sheet = write_book.get_sheet(0)
assert expense_sheet.get_name() == '支出'
 
with codecs.open(wechat_filepath, encoding='utf-8') as f:
    r = 1
    for row in csv.DictReader(f, skipinitialspace=True):
        new_row = {}
        for k in row:
            new_row[k.strip()] = row[k].strip()
        # 只记支出
        if new_row['收/支'] == '收入':
            continue
        # 退款的不用记
        if new_row['当前状态'] in ('已全额退款', '对方已退还'):
            continue
        # Jeff update，转账的需要记
        if new_row["交易类型"] in ('零钱充值'):
            continue
        expense_sheet.write(r, head.index('交易类型'), '支出')
        
        #Jeff 增加交易时间的转换，解决最后上传时日期解析问题
        trasaction_datetime = datetime.strptime(new_row['交易时间'], '%Y/%m/%d %H:%M')
        trasaction_datetime = datetime.strftime(trasaction_datetime, '%Y-%m-%d %H:%M')
        expense_sheet.write(r, head.index('日期'), trasaction_datetime)
       
        # 部分导入后部分还是要手动分类
        if new_row['交易对方']  == '深圳通':
            expense_sheet.write(r, head.index('分类'), '公共交通')
            expense_sheet.write(r, head.index('项目'), '交通费')
        elif new_row['交易对方']  == 'xx': # 请换成自己想要自动分类的
            expense_sheet.write(r, head.index('分类'), '早午晚餐')
            expense_sheet.write(r, head.index('项目'), '吃饭大事')
        elif new_row['交易对方']  == 'xxx': # 请换成自己想要自动分类的
            expense_sheet.write(r, head.index('分类'), '水果零食')
            expense_sheet.write(r, head.index('项目'), '水果零食')
        elif new_row['交易对方']  == 'xxxx': # 请换成自己想要自动分类的
            expense_sheet.write(r, head.index('分类'), '早午晚餐')
            expense_sheet.write(r, head.index('项目'), '吃饭大事')
        else:
            expense_sheet.write(r, head.index('分类'), '其他杂项')
            expense_sheet.write(r, head.index('项目'), '')
        
        expense_sheet.write(r, head.index('账户1'), '现金')
        expense_sheet.write(r, head.index('金额'), new_row['金额(元)'][1:])
        expense_sheet.write(r, head.index('商家'), new_row['交易对方'])
        expense_sheet.write(r, head.index('备注'), new_row['商品'])
        r += 1
write_book.save('wechat_output.xls')

所有的工作表： ['支出', '收入', '转账']
工作表名称：支出，行数：2，列数：11
